In [12]:
import itertools
import numpy as np
from sklearn.preprocessing import normalize
import random
import elkai
num_state = 9

def gen_cost():
    b = np.random.randint(0, 10, size=(num_state,num_state))
    cost_ = (b + b.T)
    states = [i for i in range(num_state)]
    for i in range(num_state):
        cost_[i][i] = 0
    return cost_
#print(cost_)    

def get_cost(cost, x,y):
    i1 = min(x,y)
    i2 = max(x,y)
    return cost[i1][i2]

def edge_activation(ls):
    product = np.square(ls[1]*ls[2])
    return product

def edge2_activation(ls):
    return np.square(ls[0]*ls[1]*ls[2])

def node_activation(cost, mid, others):
    #print(mid, others)
    
    if len(others) != 2:
        return 1
    middle = (get_cost(cost, mid, others[0]) + get_cost(cost, mid, others[1])) / ((np.sum(cost[mid]) - (get_cost(cost, mid, others[0]) + get_cost(cost, mid, others[1]))))
    middle *= (get_cost(cost, mid, others[0]) + get_cost(cost, mid, others[1]))
    end1 = get_cost(cost, mid, others[0]) / (np.sum(cost[others[0]]) - get_cost(cost, mid, others[0])  )
    end2 =  get_cost(cost, mid, others[1]) / (np.sum(cost[others[1]]) - get_cost(cost, mid, others[1]))
    if end1 < 0 or end2 < 0:
        print("node_act", mid, others, end1, end2, np.sum(cost[others[0]]), get_cost(cost, mid, others[0]))
        
    ls = [middle, end1, end2]
    combis = list(itertools.permutations(ls))
    prd = 1
 #   print(ls)
    k = np.square(len(combis))
    for combi in combis:
        x,y,z = combi
        prd *= x*y*z*np.exp(1/k)
    product = prd
    return product

def node_activation2(cost, mid, end):
    
    edge_cost = get_cost(cost, mid, end)
    denom1 = np.sum(cost[mid]) - get_cost(cost, mid, end)
    denom2 = np.sum(cost[end]) - get_cost(cost, mid, end)
    prod1 = edge_cost / denom1
    prod2 = edge_cost / denom2 #+ np.sum(cost[end[0]])

    product = prod1*prod2 
    return product

#print(node_activation(8,[2,7]))


In [13]:
def nbhd_triplets(cost, states, status=False, ends = []):
    combis = itertools.combinations(states, 3)
    scores = {}

    if status:
        combis = states
    
    for combi in combis:
        x,y,z = combi
   #     print(x,y,z)
        edge1 = get_cost(cost, x,y)
        edge2 = get_cost(cost, y,z)
        edge3 = get_cost(cost, x,z)
        ls = [edge1, edge2, edge3]
        ls = sorted(ls, reverse = True)
        activation = edge_activation(ls)
        if status:
            activation = edge2_activation(ls)
        if edge1 == ls[0]:
            middle = z
        elif edge2 == ls[1]:
            middle = x
        else:
            middle = y
        ls2 = []
        for s in [x,y,z]:
            if s != middle:
                ls2.append(s)
        
        if status:
            n_activation = np.cbrt(node2_activation(cost, middle, ls2, ends))
        else:
            n_activation = node_activation(cost, middle, ls2)
        if activation < 0:
            print("activation < 0", x,y,z)
        if n_activation < 0:
            print("activation < 0", x,y,z)
        score = activation*np.sqrt(n_activation)
        sstr = str(x) + str(y)+ str(z)+ str(middle)
        k = [combi]
        k.append(middle)
        scores[sstr] = score*np.exp(2/9)
    
    dictionary = [k for k, v in sorted(scores.items(), key=lambda item: item[1])]
    if not dictionary:
        return [], 0
    return dictionary[0], scores[dictionary[0]]

In [14]:
def distribute2(cost):
    dictionary = {}
    for state in range(len(cost)):
        dictionary[state] = np.sum(cost[state])
    sort = [k for k, v in sorted(dictionary.items(), key=lambda item: item[1], reverse = True)]
    heavy = sort[:5]
    light = sort[-4:]
    middle1, middle2, middle3 = [], [], []
    for i in range(len(cost)):
        rr = random.random()
        if rr <0.33:
            middle1.append(i)
        elif rr < 0.67:
            middle2.append(i)
        else:
            middle3.append(i)
#   heavy = []
    for i in range(len(cost)):
        if random.random() > 0.7:
            rr = random.random()
            if rr <0.33:
                middle1.append(i)
            elif rr <0.67:
                middle2.append(i)
            else:
                middle3.append(i)
            
    heavy = list(set(heavy))
    middle1 = list(set(middle1))
    middle2 = list(set(middle2))
    middle3 = list(set(middle3))
    res = [middle1, middle2, middle3]
    heavy.append(random.choice(np.arange(0, num_state)))
    for par in res:
        if len(par) < 3:
            par.append(random.choice(np.arange(0, num_state)))
    
    return middle1, middle2, middle3

def get_ends(string):
    middle = string[-1]
    ends = []
    for c in string:
        if c != middle:
            ends.append(int(c))
    return ends

def is_linker(a, b, edge):
    res1 = edge[0] in a and edge[1] in b
    res2 = edge[1] in b and edge[1] in a
    return res1 or res2

def get_edgess(string):
    mid = string[-1]
    edges = ""
    for c in string:
        if c != mid:
            edges = edges + c
    edges = edges[:1] + mid + edges[1:]
    return edges

In [21]:
from python_tsp.exact import solve_tsp_dynamic_programming
def process(starts, ends):
    res = {}
    for s in range(len(starts)):
        
        ends2 = starts
        ends2.pop(s)
        ends2.extend(ends)
        
        for e in range(len(ends2)):
            if starts[s] != ends2[e]:
                c = get_cost(cost,starts[s], ends2[e])
                res[str(starts[s])+str(ends2[e])] = c

    sort = [k for k, v in sorted(res.items(), key=lambda item: item[1])]
    return sort, res

def single_edge_act(num):
    return np.square(num)

def get_pairs(linker_by_cat, dictionary_by_cat, i, j):
    pairs = {}
    
    for ll in linker_by_cat[i]:
        link, score = ll, dictionary_by_cat[i][ll]
        cur = [link[0], link[1]]
        for ll2 in linker_by_cat[j]:
            link2, score2 = ll2, dictionary_by_cat[j][ll2]
            if link2[0] not in cur and link2[1] not in cur:
                n1 = node_activation2(cost, int(link[0]), int(link[1]))
                n2 = node_activation2(cost, int(link2[0]), int(link2[1]))
                pairs[link+link2] = single_edge_act(score*score2) *np.sqrt(n1*n2)*np.exp(2/9)
    if not pairs:
        return -1, 1e30, {}
    sort = [k for k, v in sorted(pairs.items(), key=lambda item: item[1])]
  #  print(pairs)
    return sort[0], pairs[sort[0]], pairs
        
def preprocess(starts, ends):
    linker_by_cat = []
    dictionary_by_cat = []
    for i in range(len(starts)):
     #   print(starts, ends)
      #  print(123, starts, ends)
        try:
            start2 = list(starts)
            start2.pop(i)
            ends2 = list(ends)
            ends2.pop(i)
        
            linkers, dictionary = process(start2, ends2)
            linker_by_cat.append(linkers)
            dictionary_by_cat.append(dictionary)
        except Exception as e:
            print(e)
            continue
    best_link, min_score, type_ = "", 1e30, ()
    scoreboard_ = {}
    for i in [(0,1), (1,2), (0,2)]:
        link, score, scoreboard = get_pairs(linker_by_cat, dictionary_by_cat, i[0], i[1])
        
        scoreboard_.update(scoreboard)
        if score < min_score:
            min_score = score
            best_link = link
            type_ = i
     #   print(23, scoreboard)
  #  print(best_link)
    scoreboard_sorted = [(k,v) for k,v in sorted(scoreboard_.items(), key=lambda item: item[1])]
    topfive = scoreboard_sorted[:6]
   # print(23,scoreboard)
    #print(23, topfive)
    return best_link, min_score, type_, topfive
    
    
    
def preconnect(ae, be, ce, best_link, t):
  #  print(t, ae, be, ce, best_link)
    if t == 0:
        res = connect(be, ce, best_link)
    elif t == 1:
        res = connect(ae, ce, best_link)
    else:
        res = connect(ae, be, best_link)
        
    return res

def connect_all(ae, be, ce, best_link, ts):
    first = preconnect(ae, be, ce, best_link[:2], ts[0])
    second = preconnect(ae, be, ce, best_link[-2:], ts[1])
    first.extend(second)
 #   print(222, best_link, len(best_link))
    link = best_link[0]
    if len(link) > 3:
        first.extend([(int(link[0]), int(link[1])), (int(link[2]), int(link[3]))])
        overall = list(set(first))
     #   print(overall)
        return overall
    else:
        return []
        
def connect(ae, be, link):
  #  print(ae, be, link)
    res = []
    res.append((int(ae[0]), int(ae[1])))
    res.append((int(ae[-2]), int(ae[-1])))
    res.append((int(be[0]), int(be[1])))
    res.append((int(be[-2]), int(be[-1])))
  #  res.append((int(link[0]), int(link[1])))
    
    return res


def parallel(reps, cost):
    best_score = 200
    best = []
    k = 1/2.714
    permutation, dp_distance = solve_tsp_dynamic_programming(cost)
    lkh = elkai.solve_int_matrix(cost)
    for i in range(reps):
        a,b,c = distribute2(cost)
        a1, score1 = nbhd_triplets(cost, a)
        b1, score2 = nbhd_triplets(cost, b)
        c1, score3 = nbhd_triplets(cost, c)
   
      #  print(a1, b1, c1)
        if not a1 or not b1 or not c1:
            continue
        
       # try:
        a2, b2, c2 = get_ends(a1), get_ends(b1), get_ends(c1)

        starts = [a2[0], b2[0], c2[0]]
       # print(a2, b2, c2)
        ends = [a2[-1], b2[-1], c2[-1]]

        try:
            best_link, link_score, type_, scoreboard = preprocess(starts, ends)
            ae, be, ce = get_edgess(a1), get_edgess(b1), get_edgess(c1)
            solns, soln_dists = [], []
            overall_score =  np.square(score1) + np.square(score2) + np.square(score3)

          #  print(123,  scoreboard)
            soln_ind = 0
            for link in scoreboard:
                
                soln_edges = connect_all(ae, be, ce, link, type_)
                is_valid, last_edge = check_if_cycle(soln_edges)
                if is_valid:
                    n1 = node_activation2(cost, last_edge[0], last_edge[1])
                    link_score = link[1]
                  #  print(23456)
                    soln_edges.append((last_edge[0], last_edge[1]))
                    soln_score = overall_score + np.square(link_score)
                    soln_score += np.square(single_edge_act(get_cost(cost, last_edge[0], last_edge[1])))#*np.sqrt(n1))
                    if soln_score > 0 :
                        solns.append(soln_edges)
                        soln_dists.append(get_dist(soln_edges))
            
            if soln_dists:
                best_index = np.argmin(soln_dists)
                edges = solns[best_index]
                sscore = soln_dists[best_index]
                if is_valid and sscore < best_score:
                    best_score = sscore
                    best = edges
        except Exception as e:
            continue
    c = 0
    cl = 0
    for i in range(len(best)):
        c += get_cost(cost, best[i][0], best[i][1])
    factor = c/dp_distance
    
    for i in range(len(lkh)-1):
        cl += get_cost(cost, lkh[i], lkh[i+1])
    cl += get_cost(cost, lkh[-1], lkh[0])
    print(dp_distance,factor,c, cl, best, permutation)
    print(lkh, c < cl)
    return best, best_score, factor

def get_dist(edges):
    c = 0
    for i in range(len(edges)):
        c += get_cost(cost, edges[i][0], edges[i][1])
    return c

def check_if_cycle(ls):
    boole = [False for i in range(num_state)]
    last = []
  #  print(ls)
    for p in ls:
        boole[p[0]] +=1
        boole[p[1]] +=1
    for i in range(len(boole)):
        if boole[i] == 1:
            last.append(i)
            boole[i] += 1
  #  print(boole)
    for i in boole:
        if i != 2:
            return False, [] 
   
    return True, last

In [22]:
factors = []
print("start")
for i in range(1000):
    cost = gen_cost()
    best, score, f = parallel(100, cost)
    if score > 0.001 and best:
        factors.append(f)
        print(np.mean(factors))
print(np.mean(factors))
#forming mini-cliques

start
47 1.297872340425532 61 47 [(1, 2), (8, 4), (4, 3), (3, 7), (6, 7), (0, 5), (1, 6), (5, 2), (0, 8)] [0, 5, 2, 4, 3, 7, 6, 1, 8]
[0, 5, 2, 4, 3, 7, 6, 1, 8] False
1.297872340425532
48 1.3333333333333333 64 48 [(2, 7), (1, 5), (3, 1), (6, 4), (0, 6), (0, 5), (8, 2), (4, 7), (3, 8)] [0, 1, 8, 2, 7, 3, 4, 6, 5]
[0, 5, 6, 4, 3, 7, 2, 8, 1] False
1.3156028368794326
41 1.3170731707317074 54 41 [(3, 8), (1, 5), (4, 6), (0, 3), (6, 7), (1, 7), (0, 8), (5, 2), (2, 4)] [0, 1, 6, 7, 5, 2, 4, 3, 8]
[0, 8, 3, 4, 2, 5, 7, 6, 1] False
1.3160929481635242
49 1.0816326530612246 53 49 [(8, 4), (3, 4), (2, 7), (5, 1), (0, 6), (2, 3), (6, 7), (8, 5), (0, 1)] [0, 1, 5, 4, 8, 7, 2, 3, 6]
[0, 1, 5, 4, 8, 7, 2, 3, 6] False
1.2574778743879493
40 1.4 56 40 [(0, 7), (2, 4), (1, 5), (8, 1), (4, 3), (8, 3), (7, 6), (0, 5), (2, 6)] [0, 1, 4, 2, 6, 7, 5, 8, 3]
[0, 3, 8, 5, 7, 6, 2, 4, 1] False
1.2859822995103596
32 1.53125 49 32 [(0, 1), (7, 4), (2, 7), (1, 8), (4, 6), (5, 6), (0, 8), (3, 5), (2, 3)] [0, 1, 3, 2

50 1.12 56 50 [(8, 4), (6, 5), (8, 1), (3, 1), (0, 3), (2, 6), (7, 5), (4, 7), (0, 2)] [0, 3, 8, 1, 5, 7, 2, 6, 4]
[0, 4, 2, 6, 7, 5, 1, 8, 3] False
1.228830407866773
53 1.0754716981132075 57 53 [(0, 7), (3, 8), (1, 5), (1, 4), (2, 6), (4, 8), (7, 5), (6, 3), (0, 2)] [0, 5, 1, 4, 8, 3, 6, 2, 7]
[0, 7, 2, 6, 3, 8, 4, 1, 5] False
1.2256354347469067
59 1.0847457627118644 64 59 [(6, 2), (0, 4), (2, 7), (1, 5), (0, 3), (6, 7), (4, 8), (5, 3), (1, 8)] [0, 3, 8, 1, 5, 6, 2, 7, 4]
[0, 3, 8, 1, 5, 6, 2, 7, 4] False
1.22276013531762
36 1.2222222222222223 44 36 [(7, 4), (2, 7), (3, 1), (5, 4), (0, 2), (8, 6), (1, 6), (8, 5), (0, 3)] [0, 3, 1, 2, 7, 4, 5, 8, 6]
[0, 3, 1, 2, 7, 4, 5, 8, 6] False
1.2227493770557123
38 1.3157894736842106 50 38 [(8, 1), (2, 3), (0, 2), (8, 3), (0, 5), (1, 6), (7, 5), (4, 7), (4, 6)] [0, 2, 4, 7, 6, 1, 3, 8, 5]
[0, 5, 8, 3, 1, 6, 7, 4, 2] False
1.2245736926758788


KeyboardInterrupt: 

In [ ]:
np.mean(factors)

In [ ]:
test = [(8, 1), (0, 3), (6, 4), (5, 7), (4, 2), (0, 5), (7, 2), (1, 3), (6, 8)] 
def no_cycle(test):
    boole = [0 for i in range(num_state)]
    for e in test:
        boole[e[0]] += 1
        boole[e[1]] += 1
    for i in boole:
        
print(no_cycle(test))